In [0]:
#kaggle datasets download -d karakaggle/kaggle-cat-vs-dog-dataset

In [0]:
! pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"smagni","key":"0ea22017b6052eb6b8adc0ef5988d4dc"}'}

In [0]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d karakaggle/kaggle-cat-vs-dog-dataset

 98% 770M/787M [00:15<00:00, 55.1MB/s]
100% 787M/787M [00:15<00:00, 52.9MB/s]


In [0]:
!unzip /content/kaggle-cat-vs-dog-dataset.zip

# START

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from keras.preprocessing import image

Using TensorFlow backend.


In [0]:
path = '/content/kagglecatsanddogs_3367a/PetImages/'
X = []
y = []

In [0]:
def prep(path, label) :
    img = image.load_img(path, target_size= (100, 100))
    img = image.img_to_array(img)
    X.append(img)
    y.append(label)

In [0]:
for folder in os.listdir(path):
    for img in tqdm(os.listdir(path+folder)) :
        final_path = path+folder+'/'+img
        if final_path.endswith(('jpg', 'png')) :
            prep(final_path, folder)            

In [0]:
len(X), len(y)

(24959, 24959)

In [0]:
X = np.array(X)
y = np.array(y)

In [0]:
X.shape, y.shape

((24959, 100, 100, 3), (24959,))

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [0]:
X_train.shape, X_test.shape

((18719, 100, 100, 3), (6240, 100, 100, 3))

In [0]:
from sklearn.preprocessing import LabelEncoder
le_y = LabelEncoder()
y_train = le_y.fit_transform(y_train)
y_test = le_y.transform(y_test)

# NN

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Flatten

In [0]:
clf = Sequential()

In [0]:
clf.add(Conv2D(32, (3, 3), activation= 'relu', input_shape = (100, 100, 3)))
clf.add(Conv2D(32, (5, 5), activation= 'relu'))
clf.add(MaxPool2D(pool_size= (2,2)))
clf.add(Dropout(rate = 0.25))

In [0]:
clf.add(Conv2D(32, (3, 3), activation= 'relu'))
clf.add(Conv2D(16, (5, 5), activation= 'relu'))
clf.add(MaxPool2D(pool_size= (2,2)))
clf.add(Dropout(rate = 0.25))

In [0]:
clf.add(Flatten())

In [0]:
clf.add(Dense(64, activation= 'relu'))

In [0]:
clf.add(Dense(1, activation = 'sigmoid'))

In [0]:
clf.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [0]:
clf.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 98, 98, 32)        896       
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 94, 94, 32)        25632     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 47, 47, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 47, 47, 32)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 45, 45, 32)        9248      
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 41, 41, 16)        12816     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 20, 20, 16)       

In [0]:
clf.fit(X_train, y_train, batch_size= 128, epochs= 10, validation_split= 0.2)

Train on 14975 samples, validate on 3744 samples
Epoch 1/10
14975/14975 [==============================] - 18s 1ms/step - loss: 0.6540 - accuracy: 0.5974 - val_loss: 0.6828 - val_accuracy: 0.5761
Epoch 2/10
14975/14975 [==============================] - 18s 1ms/step - loss: 0.6448 - accuracy: 0.6141 - val_loss: 0.6676 - val_accuracy: 0.5911
Epoch 3/10
14975/14975 [==============================] - 18s 1ms/step - loss: 0.6323 - accuracy: 0.6275 - val_loss: 0.6684 - val_accuracy: 0.6031
Epoch 4/10
14975/14975 [==============================] - 18s 1ms/step - loss: 0.6152 - accuracy: 0.6440 - val_loss: 0.6700 - val_accuracy: 0.6010
Epoch 5/10
14975/14975 [==============================] - 18s 1ms/step - loss: 0.6013 - accuracy: 0.6607 - val_loss: 0.6945 - val_accuracy: 0.5948
Epoch 6/10
14975/14975 [==============================] - 18s 1ms/step - loss: 0.5811 - accuracy: 0.6822 - val_loss: 0.6758 - val_accuracy: 0.6018
Epoch 7/10
14975/14975 [==============================] - 18s 1ms/ste

In [0]:
clf.evaluate(X_test, y_test)

6240/6240 [==============================] - 4s 571us/step


[0.679724262158076, 0.6496794819831848]

In [0]:
y_pred = clf.predict(X_test)

In [0]:
y_pred = y_pred >= 0.5

In [0]:
from sklearn.metrics import confusion_matrix, classification_report

In [0]:
print(confusion_matrix(y_pred, y_test))

[[2089 1163]
 [1023 1965]]


In [0]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.64      0.67      0.66      3112
           1       0.66      0.63      0.64      3128

    accuracy                           0.65      6240
   macro avg       0.65      0.65      0.65      6240
weighted avg       0.65      0.65      0.65      6240

